In [1]:
import os
import json

os.environ["OPENPI_DATA_HOME"] = (
    "/mnt/virtual_ai0001071-01239_SR006-nfs2/apanasevich/openpi/assets"
)
os.environ["HF_HOME"] = "/mnt/virtual_ai0001071-01239_SR006-nfs2/.cache/huggingface"
os.environ["XDG_CACHE_HOME"] = "/mnt/virtual_ai0001071-01239_SR006-nfs2/.cache"


from lerobot.common.datasets.create_dataloader import create_lerobot_dataloader
from lerobot.common.datasets.data_config import (
    LeRobotAgibotTwoFingerDataConfig,
    LeRobotAgibotDexHandDataConfig,
)
from rich_argparse import RichHelpFormatter
import argparse

import torch

from accelerate.utils import InitProcessGroupKwargs
from datetime import timedelta

from lerobot.common.datasets.data_config import (
    AssetsConfig as LeRobotAssetsConfig
)
from lerobot.common.datasets.data_config import DataConfig as LeRobotBaseDataConfig
from lerobot.common.policies.pi0.configuration_pi0 import PI0Config
from lerobot.common.policies.pi0fast.configuration_pi0fast import PI0FASTConfig
from tqdm import tqdm
from accelerate import Accelerator
from lerobot.common.utils.normalize import RunningStats, save as save_stats
import torch.distributed as dist
from pathlib import Path
from omegaconf import OmegaConf
from hydra.utils import instantiate
from lerobot.common.datasets.create_dataloader import create_lerobot_dataset_by_config
from torch.utils.data import DataLoader


def get_data(
    dataset_config_path: str, 
    assets_dir: str, 
    action_horizon: int | None = None,
    action_dim: int | None = None
):
    cfg = OmegaConf.load(dataset_config_path)
    cfg = OmegaConf.to_container(cfg, resolve=True)

    if action_horizon is not None:
        cfg['action_horizon'] = action_horizon
    
    data_config = instantiate(cfg)
    model_cfg = PI0Config()
    lerobot_dataset = create_lerobot_dataset_by_config(
        data_config_factory=data_config,
        model_config=model_cfg,
        assets_dirs=assets_dir,
        normalization_mode="mean_std", #it does not matter
        skip_norm_stats=True,
        skip_model_transforms=True,
        return_norm_stats=False,
        )
    
    class InnerDataset(torch.utils.data.Dataset):
        def __init__(self, dataset, action_horizon, action_dim):
            self.dataset = dataset
            self.action_horizon = action_horizon
            self.action_dim = action_dim
            
            if self.action_dim is None:
                self.action_dim = self.dataset[0]['actions'].shape[1]

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            return self.dataset[idx]['actions'][:self.action_horizon, :self.action_dim]

    return InnerDataset(lerobot_dataset, action_horizon, action_dim), cfg['repo_id']

/home/jovyan/shares/SR006.nfs2/micromamba/envs/af_beast_calvin/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/virtual_ai0001071-01239_SR006-nfs2/afedorov/projects/constant_repos/lerobot-fork/lerobot/common/datasets/vlm_datasets/qwen_dataset_specific_augmentations.py:147: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.2),
/mnt/virtual_ai0001071-01239_SR006-nfs2/afedorov/projects/constant_repos/lerobot-fork/lerobot/common/datasets/vlm_datasets/qwen_dataset_specific_augmentations.py:149: UserWarning: Argument(s) 'shift_limit' are not valid for transform OpticalDistortion
  A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=0.2),
/home/jovyan/shares/SR006.nfs2/micromamba/envs/af_beast_calvin/lib

In [2]:
data_config = "/mnt/virtual_ai0001071-01239_SR006-nfs2/afedorov/projects/constant_repos/lerobot-fork/lerobot/conf/robotics_dataset/individual/aloha_aij.yaml"
assets_dir = "/mnt/virtual_ai0001071-01239_SR006-nfs2/afedorov/assets"

dataset, config_name = get_data(data_config, assets_dir, action_horizon=16, action_dim=14)

# with open(f'{assets_dir}/{config_name}/norm_stats.json') as f:
with open(f'{assets_dir}/aloha_aij_delta/norm_stats.json') as f:
    norm_stats = json.load(f)
max_stats = norm_stats['norm_stats']['actions']['q99']
min_stats = norm_stats['norm_stats']['actions']['q01']

Creating LeRobot dataset for repo_id: aloha_aij

Initialized PromptFromLeRobotTaskTorch with 1858 tasks.

map_to_unified_space: False

In [3]:
import torch
import numpy as np
%matplotlib inline

from beast.bspline_tokenizer import BSpline_Tokenizer

num_indexes = np.random.randint(0, len(dataset), size=10_000)

traj = np.stack([dataset[i].numpy() for i in num_indexes])
num_dof = traj.shape[-1]
seq_len = traj.shape[-2]

action_min = np.array(min_stats[:num_dof])
action_max = np.array(max_stats[:num_dof])

tokenizer = BSpline_Tokenizer(num_dof=num_dof, num_basis=50, seq_len=seq_len, gripper_zero_order=False, gripper_indices=[]).to('cuda')

trajs = torch.from_numpy(traj).to('cuda')
action_min = torch.from_numpy(action_min).to('cuda')
action_max = torch.from_numpy(action_max).to('cuda')

### normalize the trajectory before tokenization
trajs = (trajs - action_min) / (action_max - action_min)
### tokenize the trajectory
tokenizer.visualize_reconstruction_error(trajs)


/mnt/virtual_ai0001071-01239_SR006-nfs2/afedorov/projects/beast_tokenizer/beast/bspline_tokenizer.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float32):


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.84 GiB. GPU 0 has a total capacity of 79.21 GiB of which 5.28 GiB is free. Process 1675116 has 73.04 GiB memory in use. Including non-PyTorch memory, this process has 860.00 MiB memory in use. Of the allocated memory 164.30 MiB is allocated by PyTorch, and 17.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trajs.shape, action_min.shape

(torch.Size([3, 16, 14]), torch.Size([16]))

In [ ]:
plt.plot(np.arange(100), np.random.randn(100))
plt.show()

NameError: name 'plt' is not defined